## AIONER Benchmarks

In this notebook, we will perform benchmarks with AIONER. To do this, we will start from the original repository of [AIONER](https://github.com/ncbi/AIONER), to which several modifications were made. These modifications can be found in this [forked repository](https://github.com/manumartinm/AIONER).

Next, we will load the data in PubTator format, and finally, we will obtain the results along with the evaluation metrics, which we will save in a JSON file.

### Libraries

In [ ]:
from datasets_utils import datasets_map
from ner_benchmark_data_wrangler import NERBenchmarkDataWrangler
import subprocess
import os

### Data Loading

We are going to load the training, validation, and test data in PubTator format into the `custom_datasets` directory. We will also obtain the vocabulary and save it in the `custom_vocab` directory.

In [ ]:

metrics = []

for dataset in list(datasets_map.keys()):
    wrangler = NERBenchmarkDataWrangler(dataset, datasets_map)
    train_df = wrangler.get_format_data('train')

    valid_df = wrangler.get_format_data('valid')
    test_df = wrangler.get_format_data('test')

    train_pubtator = wrangler.to_pubtator('train')
    test_pubtator = wrangler.to_pubtator('test')
    valid_pubtator = wrangler.to_pubtator('valid')
    labels = train_df['labels'].unique().tolist() + valid_df['labels'].unique().tolist() + test_df['labels'].unique().tolist()
    labels = list(set(labels))

    join_texts = train_pubtator + valid_pubtator + test_pubtator

    if not os.path.exists(f'./custom_datasets/{dataset}'):
        os.makedirs(f'./custom_datasets/{dataset}')

    if not os.path.exists(f'./custom_vocab/{dataset}'):
        os.makedirs(f'./custom_vocab/{dataset}')

    with open(f'./custom_vocab/{dataset}.txt', 'w') as f:
        f.write('\n'.join(labels))

    with open(f'./custom_datasets/{dataset}/train.txt', 'w') as f:
        f.write('\n'.join(train_pubtator))

    with open(f'./custom_datasets/{dataset}/valid.txt', 'w') as f:
        f.write('\n'.join(valid_pubtator))

    with open(f'./custom_datasets/{dataset}/test.txt', 'w') as f:
        f.write('\n'.join(test_pubtator))

    subprocess.run(
        f'python AIONER/src/Format_Preprocess.py -i ../custom_datasets/{dataset}/ -o ../finetunning_files/{dataset}/',
        shell=True
    )

    subprocess.run(
      f'python AIONER/src/AIONER_FineTune.py -t ./finetunning_files/{dataset}.conll -m AIONER/pretrained_models/AIONER/Bioformer-softmax-AIONER.h5 -v ./custom_vocab/{dataset}.txt -e ALL -o finetune/{dataset}/',
      shell=True
    )

### Obtaining Results and Metrics

Finally, we will obtain the results using the script provided by AIONER, which is `AIONER_Run.py`, and we will calculate the evaluation metrics with the `Test_Evaluation.py` script. The metrics will be saved in the `custom_metrics` directory.

In [ ]:
for dataset in list(datasets_map.keys()):
    subprocess.run(
      f'python AIONER/src/AIONER_Run.py -i ./custom_datasets/{dataset}/ -m finetune/{dataset}/Bioformer-softmax-AIONER.h5 -v ./custom_vocab/{dataset}.txt -e ALL -o output/{dataset}/',
      shell=True
    )

    subprocess.run(
      f'python AIONER/src/Test_Evaluation.py --gold ./custom_datasets/{dataset}/texts.txt --pred ./output/{dataset}/texts.txt -o ./metrics/{dataset}.json',
      shell=True
    )